In [6]:
import pandas as pd

# Lee el archivo CSV
df = pd.read_csv('jamb_exam_results.csv')

# Muestra las primeras filas del DataFrame
print(df.head())

   JAMB_Score  Study_Hours_Per_Week  Attendance_Rate  Teacher_Quality  \
0         192                    22               78                4   
1         207                    14               88                4   
2         182                    29               87                2   
3         210                    29               99                2   
4         199                    12               98                3   

   Distance_To_School School_Type School_Location Extra_Tutorials  \
0                12.4      Public           Urban             Yes   
1                 2.7      Public           Rural              No   
2                 9.6      Public           Rural             Yes   
3                 2.6      Public           Urban              No   
4                 8.8      Public           Urban              No   

  Access_To_Learning_Materials Parent_Involvement IT_Knowledge  Student_ID  \
0                          Yes               High       Medium      

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor